In [65]:
import numpy as np
import pandas as pd
import nasdaqdatalink, math

from sklearn import preprocessing, svm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [3]:
api_key = "NxWhepjVp5jL9TyShexE"

# df = quandl.get('WIKI/GOOGL', authtoken=api_key)
data = nasdaqdatalink.get('WIKI/GOOGL')

In [4]:
data.head()

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2004-08-19,100.01,104.06,95.96,100.335,44659000.0,0.0,1.0,50.159839,52.191109,48.128568,50.322842,44659000.0
2004-08-20,101.01,109.08,100.50,108.310,22834300.0,0.0,1.0,50.661387,54.708881,50.405597,54.322689,22834300.0
2004-08-23,110.76,113.48,109.05,109.400,18256100.0,0.0,1.0,55.551482,56.915693,54.693835,54.869377,18256100.0
2004-08-24,111.24,111.60,103.57,104.870,15247300.0,0.0,1.0,55.792225,55.972783,51.945350,52.597363,15247300.0
2004-08-25,104.76,108.00,103.88,106.000,9188600.0,0.0,1.0,52.542193,54.167209,52.100830,53.164113,9188600.0


In [5]:
data.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Ex-Dividend', 'Split Ratio',
       'Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close', 'Adj. Volume'],
      dtype='object')

In [6]:
df = data[['Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close', 'Adj. Volume']]

In [7]:
df['HL_PCT'] = ((df['Adj. High'] - df['Adj. Close']) / df['Adj. Close']) * 100.0

df['PCT_Change'] = ((df['Adj. Close'] - df['Adj. Open']) / df['Adj. Open']) * 100.0

C:\Users\hemanth.kotapro\AppData\Local\Temp\ipykernel_11540\2610305750.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['HL_PCT'] = ((df['Adj. High'] - df['Adj. Close']) / df['Adj. Close']) * 100.0
C:\Users\hemanth.kotapro\AppData\Local\Temp\ipykernel_11540\2610305750.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PCT_Change'] = ((df['Adj. Close'] - df['Adj. Open']) / df['Adj. Open']) * 100.0


In [9]:
df.columns

Index(['Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close', 'Adj. Volume',
       'HL_PCT', 'PCT_Change'],
      dtype='object')

In [11]:
df = df[['Adj. Close', 'HL_PCT','PCT_Change','Adj. Volume']]

In [12]:
df.head()

,Adj. Close,HL_PCT,PCT_Change,Adj. Volume
Date,,,,
2004-08-19,50.322842,3.712563,0.324968,44659000.0
2004-08-20,54.322689,0.710922,7.227007,22834300.0
2004-08-23,54.869377,3.729433,-1.227880,18256100.0
2004-08-24,52.597363,6.417469,-5.726357,15247300.0
2004-08-25,53.164113,1.886792,1.183658,9188600.0


In [13]:
df.describe()

,Adj. Close,HL_PCT,PCT_Change,Adj. Volume
count,3424.000000,3424.000000,3424.000000,3.424000e+03
mean,409.057885,1.064055,-0.025030,7.818568e+06
std,257.773495,1.134349,1.505326,8.248211e+06
min,50.159839,0.000000,-9.179757,5.211410e+05
25%,231.316552,0.335169,-0.742241,2.430647e+06
50%,300.264387,0.728061,-0.016254,5.076200e+06
75%,561.200000,1.391731,0.752556,1.020910e+07
max,1187.560000,11.157480,8.759770,8.215110e+07


In [73]:
forecast_col = 'Adj. Close'

forecast_out = int(math.ceil(0.01*len(df)))

In [74]:
df['label'] = df[forecast_col].shift(-forecast_out) 

In [75]:
df.dropna(inplace=True)

In [76]:
df.head()

,Adj. Close,HL_PCT,PCT_Change,Adj. Volume,label
Date,,,,,
2004-08-19,50.322842,3.712563,0.324968,44659000.0,66.495265
2004-08-20,54.322689,0.710922,7.227007,22834300.0,67.739104
2004-08-23,54.869377,3.729433,-1.227880,18256100.0,69.399229
2004-08-24,52.597363,6.417469,-5.726357,15247300.0,68.752232
2004-08-25,53.164113,1.886792,1.183658,9188600.0,69.639972


In [77]:
df.tail()

,Adj. Close,HL_PCT,PCT_Change,Adj. Volume,label
Date,,,,,
2016-04-19,776.25,1.893720,-1.802657,2197615.0,748.46
2016-04-20,774.92,0.611676,-0.544176,1713088.0,744.27
2016-04-21,780.00,0.215372,0.346065,3881420.0,735.86
2016-04-22,737.77,2.189029,-0.825369,7039948.0,730.06
2016-04-25,742.21,0.359736,0.932889,2472061.0,731.09


In [78]:
df.describe()

,Adj. Close,HL_PCT,PCT_Change,Adj. Volume,label
count,2941.000000,2941.000000,2941.000000,2.941000e+03,2941.000000
mean,328.392061,1.125148,-0.027228,8.824079e+06,335.239704
std,169.159256,1.174445,1.574343,8.481979e+06,171.552355
min,50.159839,0.000000,-9.179757,5.211410e+05,66.495265
25%,216.668836,0.365256,-0.822222,3.645300e+06,220.480602
50%,280.917164,0.787485,-0.027505,6.035700e+06,284.076918
75%,433.658663,1.494575,0.798613,1.156780e+07,439.892907
max,793.960000,11.157480,8.759770,8.215110e+07,793.960000


In [79]:
X = np.array(df.drop(['label'], 1))
y = np.array(df['label'])

C:\Users\hemanth.kotapro\AppData\Local\Temp\ipykernel_11540\3704626254.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = np.array(df.drop(['label'], 1))


In [80]:
X = preprocessing.scale(X)
# X = X[:-forecast_out+1]

df.dropna(inplace=True)

y = np.array(df['label'])

In [81]:
X_train,X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

In [91]:
clf = LinearRegression()
clf.fit(X_train,y_train)

accuracy = clf.score(X_test, y_test)

In [92]:
print(accuracy,forecast_out)

0.9657097423016403 30
